IAMSI -- 2017-2018
--------
*&copy;Equipe pédagogique IAMSI'2018*


# TME 09 : Règles d'association

<font size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

*Double-cliquer ici et insérer les noms et prénoms de votre binôme*

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce fichier ipython complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un second envoi peut être fait **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyez le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

#### A télécharger

Avant de commencer, télécharger l'archive **tme-09.tgz** à l'adresse indiquée.

Une fois détarée, cette archive crée un répertoire contenant :
- deux fichiers exécutables (Linux): *apriori32* et *fpgrowth32* (ces 2 programmes ont été développés par Christian Borgelt, chercheur au ``European Center of Soft Computing" , plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance.
- un répertoire *datasets* contenant des bases d'exemples pour tester les algorithmes
- ce fichier ipython notebook que vous lisez en ce moment et que vous allez compléter.



### Imports utilisés

In [26]:
from functools import reduce

## Traitement d'une base d'apprentissage

### Chargement de la base

On commence par travailler sur la base exemple du fichier "*exemple-1.txt*" (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [1]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font size="+1">**[Q]**</font> Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble d'items plutôt que comme une liste. Il sera ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [5]:
def chargeBase(fichier):
    """ Prends en argument un fichier et renvoie un dictionnaire 
        avec comme clé les numéros de ligne et les valeurs le set des itemsets
        
        :param fichier: fichier en entrée contenant des transactions
        :return: BASE
    """
    
    with open('datasets/exemple-1.txt', 'r') as fichier:
        lecteur = csv.reader(fichier, delimiter=' ')
        return {i: set(itemsets) for i, itemsets in enumerate(lecteur)}

In [6]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{0: {'a', 'b', 'c'},
 1: {'a', 'd', 'e'},
 2: {'b', 'c', 'd'},
 3: {'a', 'b', 'c', 'd'},
 4: {'b', 'c'},
 5: {'a', 'b', 'd'},
 6: {'d', 'e'},
 7: {'a', 'b', 'c', 'd'},
 8: {'c', 'd', 'e'},
 9: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font size="+1">**[Q]**</font> Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [27]:
def noms_items(BASE):
    """ Fonction qui prends en argument une BASE et rend 
        l'ensemble des items qui la compose 
        
        :param BASE: dict of sets
        :return: ensemble d'itemsets
    """
    return reduce((lambda x, y: x | y), BASE.values())

In [28]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font size="+1">**[Q]**</font> Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

Remarque: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [37]:
def singletons(BASE):
    return [set(itemsets) for itemsets in noms_items(BASE)]

In [38]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'a'}, {'c'}, {'b'}, {'e'}, {'d'}]

<font size="+1">**[Q]**</font> Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [85]:
def comptage(BASE, itemset):
    comptage = 0
    res_boolean = [list(map(lambda x: x in transaction, itemset)) 
                   for transaction in BASE.values()]
    comptage = sum([reduce(lambda x, y: x and y, element) for element in res_boolean])
        
    return comptage

In [86]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font size="+1">**[Q]**</font> Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [87]:
def support(BASE, itemset):
    return comptage(BASE, itemset)/len(BASE.keys())

In [88]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : "+str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [135]:
def apriori_gen(itemsets):
    k = len(itemsets[0])
    res, i = [], 0
    for itemset in itemsets:
        for item in itemsets[i:]:            
            st = set(item) | itemset
            if len(st) == k+1:
                res = list(filter(lambda x: len(x & st) != k+1, res))
                res.append(st)
                
    return res

In [156]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'a', 'c'}, {'b', 'c'}, {'a', 'd'}, {'b', 'd'}, {'c', 'd'}]

Exemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend 
[{'b', 'a', 'd'}, {'b', 'a', 'c'}, {'a', 'c', 'd'}, {'b', 'c', 'd'}]


[]

<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [176]:
def apriori(BASE, val):
    single = singletons(BASE)
    reduce(lambda x: apriori_gen(x[-1]), [single])
    found_itemsets = apriori_gen(single) + single
    return [(itemset, support(BASE, itemset)) for itemset in found_itemsets]

In [177]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'a', 'c'}, 0.4), ({'b', 'a'}, 0.5), ({'b', 'c'}, 0.6), ({'a', 'e'}, 0.1), ({'c', 'e'}, 0.1), ({'b', 'e'}, 0.0), ({'a', 'd'}, 0.4), ({'c', 'd'}, 0.4), ({'b', 'd'}, 0.4), ({'e', 'd'}, 0.3), ({'a'}, 0.6), ({'c'}, 0.7), ({'b'}, 0.7), ({'e'}, 0.3), ({'d'}, 0.7)]


## Expérimentations

### Vérification avec la base du TD

Utilisez votre fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que vous obtenez les bons itemsets fréquents.

In [17]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'c'}, 0.5), ({'a'}, 0.5), ({'e'}, 0.6), ({'b'}, 0.7), ({'d'}, 0.9), ({'c', 'b'}, 0.3), ({'c', 'd'}, 0.4), ({'a', 'e'}, 0.4), ({'a', 'b'}, 0.3), ({'a', 'd'}, 0.4), ({'a', 'e'}, 0.4), ({'b', 'e'}, 0.4), ({'d', 'e'}, 0.6), ({'c', 'b'}, 0.3), ({'a', 'b'}, 0.3), ({'e', 'b'}, 0.4), ({'d', 'b'}, 0.6), ({'c', 'd'}, 0.4), ({'a', 'd'}, 0.4), ({'d', 'e'}, 0.6), ({'d', 'b'}, 0.6), ({'a', 'd', 'e'}, 0.4), ({'a', 'd', 'e'}, 0.4), ({'a', 'd', 'e'}, 0.4), ({'a', 'd', 'e'}, 0.4), ({'b', 'd', 'e'}, 0.4)]


### Comparaisons avec les implémentations apriori32 et fpgrowth

<font size="+1">**[Q]**</font> Tester votre programme sur les 3 bases données dans datasets et confronter vos résultats avec le programme *apriori32*.

<font size="+1">**[Q]**</font> Utiliser le programme *apriori32* pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

### Utilisation de fp-growth

<font size="+1">**[Q]**</font>  Utiliser *fpgrowth32* pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

<font size="+1">**[Q]**</font> En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des programmes *apriori32* et *fpgrowth32*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori32*, soit *fpgrowth32*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori32 -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth32 -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
